In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math
from pathlib import Path

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier

from collections import defaultdict

In [9]:
df = pd.read_csv("analysis/not_uploaded/IA/AVA/.scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-9.pth.txt")
df["img"] = df["img"].apply(lambda row: int(Path(row).stem))
df["score"] = df["score"].apply(lambda row: eval(row)[0])

with open("analysis/not_uploaded/IA/AVA/style_image_lists/styles.txt") as f:
    styles_dict = [(int(line.split(" ")[0]), line.split(" ")[1].strip()) for line in f.readlines()]
    styles_dict = dict(styles_dict)

with open("analysis/not_uploaded/IA/AVA/style_image_lists/train.jpgl") as f:
    train_files = [int(line) for line in f.readlines()]

with open("analysis/not_uploaded/IA/AVA/style_image_lists/train.lab") as f:
    train_label = [int(line) for line in f.readlines()]

train_files = defaultdict(lambda: -1, zip(train_files, train_label))

with open("analysis/not_uploaded/IA/AVA/style_image_lists/test.jpgl") as f:
    test_files = [int(line) for line in f.readlines()]

with open("analysis/not_uploaded/IA/AVA/style_image_lists/test.multilab") as f:
    test_label = [line.strip().split(" ") for line in f.readlines()]
    test_label = [[int(lab) for lab in line] for line in test_label]

test_files = defaultdict(lambda: [], zip(test_files, test_label))

df = df[df["img"].isin(train_files) | df["img"].isin(test_files)]
df["mode"] = df["img"].apply(lambda row: "train" if row in train_files else "test")

df["train_label"] = df["img"].apply(lambda row: train_files[row])
df["test_label"] = df["img"].apply(lambda row: test_files[row])

df

,img,score,styles_contrast_strength,styles_brightness_strength,styles_saturation_strength,styles_shadows_strength,styles_highlights_strength,styles_exposure_strength,styles_vibrance_strength,styles_temperature_strength,...,technical_impulse_noise_strength,composition_rotate_strength,composition_hcrop_strength,composition_vcrop_strength,composition_leftcornerscrop_strength,composition_rightcornerscrop_strength,composition_ratio_strength,mode,train_label,test_label
13,483560,0.750488,0.933105,-0.280273,0.634766,0.163818,-0.031433,0.588379,0.148560,0.008797,...,-0.476562,0.314941,0.131592,-0.487305,-0.405518,-0.453857,0.551758,train,4,[]
23,423049,0.797363,0.363770,-0.072754,0.207764,0.060028,0.218628,-0.020828,0.101746,-0.206055,...,-0.008965,0.140747,0.035858,0.154053,0.151855,0.185669,0.343018,test,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
37,952321,0.142822,-0.037628,0.352783,-0.291504,-0.133179,0.366699,0.163696,0.081299,0.009987,...,0.241821,-0.046051,-0.082642,0.098938,0.081299,0.118469,-0.101624,test,-1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
92,385949,0.846680,-0.004475,-0.263916,0.129883,-0.334229,-0.134644,-0.057343,0.106140,0.201294,...,-0.268799,-0.030014,-0.016388,0.352295,0.393066,0.367676,0.243408,test,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
106,58524,0.655762,-0.074524,0.283203,0.021744,-0.108276,0.452148,0.138794,0.100037,0.376221,...,0.029556,-0.010918,-0.092041,0.133911,0.097229,0.137085,0.512207,train,11,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255362,177394,0.032043,-0.034271,-0.023834,0.016556,-0.616211,0.640625,0.052795,0.132080,-0.113037,...,0.396240,0.047424,-0.073547,-0.528320,-0.555664,-0.510742,0.162842,train,12,[]
255402,924187,0.281738,0.132080,-0.282227,0.540039,-0.294922,0.197754,-0.185059,0.214355,0.593262,...,0.131592,-0.108521,0.081055,-0.335938,-0.354980,-0.340332,0.713867,train,8,[]
255454,924217,0.427490,-0.777832,0.190308,0.673828,-0.483154,0.187988,-0.424316,0.273682,-0.306152,...,0.062805,-0.164062,0.178711,0.125488,0.119019,0.079956,1.268555,train,8,[]
255456,458455,0.935547,0.487061,-0.323975,0.080139,0.029434,0.366455,-0.194336,0.068787,-0.368896,...,-0.235840,-0.182983,0.391113,-0.291260,-0.242188,-0.285156,0.054230,test,-1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]"


In [10]:
features = [c for c in df.columns if c not in ["index", "img"]]
reg_df = df.dropna()
train = reg_df[reg_df["mode"]=="train"].reset_index()[features]
test = reg_df[reg_df["mode"]=="test"].reset_index()[features]
features

['score',
 'styles_contrast_strength',
 'styles_brightness_strength',
 'styles_saturation_strength',
 'styles_shadows_strength',
 'styles_highlights_strength',
 'styles_exposure_strength',
 'styles_vibrance_strength',
 'styles_temperature_strength',
 'styles_tint_strength',
 'technical_jpeg_compression_strength',
 'technical_defocus_blur_strength',
 'technical_motion_blur_strength',
 'technical_pixelate_strength',
 'technical_gaussian_noise_strength',
 'technical_impulse_noise_strength',
 'composition_rotate_strength',
 'composition_hcrop_strength',
 'composition_vcrop_strength',
 'composition_leftcornerscrop_strength',
 'composition_rightcornerscrop_strength',
 'composition_ratio_strength',
 'mode',
 'train_label',
 'test_label']

In [46]:
reg = OneVsRestClassifier(svm.LinearSVC(class_weight='balanced'), n_jobs=-1).fit(train.drop(columns=["mode", "train_label", "test_label"]), train["train_label"])
reg.score(train.drop(columns=["mode", "train_label", "test_label"]), train["train_label"])

0.20275066548358472

In [51]:
preds = []
for est in reg.estimators_:
    preds.append(est.predict(test.drop(columns=["mode", "train_label", "test_label", "pred", "y_pred"])))

In [52]:
y_pred = [[] for _ in range(len(preds[0]))]
for est in preds:
    for i, row in enumerate(est):
        y_pred[i].append(row)
test["y_pred"] = y_pred

In [12]:
test["pred"] = reg.predict(test.drop(columns=["mode", "train_label", "test_label"]))

In [53]:
test[["test_label", "y_pred"]]

,test_label,y_pred
0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0]"
1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0]"
3,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]"
4,"[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1]"
...,...,...
2804,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0]"
2805,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0]"
2806,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0]"
2807,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1]"


In [7]:
test["found"] = test.apply(lambda row: row.test_label[row.pred-1], axis=1)

In [8]:
sum(list(test["found"]))/len(list(test["found"]))

0.25311498754004985

In [83]:
test["pred"] = clf.predict(test.drop(columns=["mode", "train_label", "test_label", "pred", "found"]))

In [88]:
probs = clf.predict_proba(test.drop(columns=["mode", "train_label", "test_label", "pred", "found"]))

In [90]:
probs2 = [[1 if score > 1/14 else 0 for score in i] for i in probs]

In [85]:
test[["test_label", "pred"]]

,test_label,pred
0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",8
1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",1
3,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2
4,"[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",11
...,...,...
2804,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",9
2805,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",14
2806,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",12
2807,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",10


In [60]:
precs = []
for i in range(14):
    precs.append(sklearn.metrics.precision_score(test["test_label"].apply(lambda row: row[i]), test["y_pred"].apply(lambda row: row[i])))
    print(precs[-1])
print("avg",sum(precs) / len(precs))

0.14149139579349904
0.2775476387738194
0.06673960612691467
0.0988909426987061
0.22759856630824374
0.11160318866253321
0.2491258741258741
0.07245155855096883
0.09014869888475836
0.13958810068649885
0.28940568475452194
0.15857605177993528
0.08297320656871218
0.07001044932079414
avg 0.14829649735969855


In [86]:
test["found"] = test.apply(lambda row: row.test_label[row.pred-1], axis=1)

In [87]:
sum(list(test["found"]))/len(list(test["found"]))

0.2121751512993948